# 1. car 데이터셋을 설치한다.

https://public.roboflow.com/object-detection/vehicles-openimages

In [ ]:
%mkdir /content/car/
%cd /content/car/
!curl -L "https://public.roboflow.com/ds/Uw5hJ2lRkV?key=mS9tCpXpkE" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

# 2. YOLOV5 레포지토리 복사 후에 클론 생성

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git 

## yolov5 디렉토리로 이동후 yolov5를 위한 패키지 설치

In [ ]:
%cd /content/yolov5/
%pip install -r requirements.txt

## 경로 확인

In [ ]:
%cat /content/car/data.yaml

# 3. train과 valid 갯수 확인

In [ ]:
%cd /
from glob import glob

img_train_list=glob('/content/car/train/images/*.jpg')
img_val_list=glob('/content/car/valid/images/*.jpg')

print(len(img_train_list), len(img_val_list))

## 4. 화면의 좌측에서 data.yaml을 수정하거나, 미리 수정한 pistol.yaml을 pistol 아래에 설치한다.

## train: /content/dataset/train/images
## val: /content/dataset/valid/images

## nc: 5
## names: ['Ambulance', 'Bus', 'Car', 'Motorcycle', 'Truck']

## data.yaml의 경로를 확인

In [ ]:
%cat /content/car/data.yaml

# 5. 학습하기

###    car_results 폴더에 학습한 모델을 저장

- data : 데이터셋의 경로정보를 가진 yaml 화일 설정
- cfg : 학습에 사용할 모델(yolov5s.pt)의 설정과 관련한 화일(yolov5s.yaml) 지정
- weights : 학습에 사용할 모델(yolov5s.pt) 지정

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 1 --data /content/car/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name car_results --cache

## 모델 생성 결과 확인

In [11]:
!ls /content/yolov5/runs/train/car_results3/weights

best.pt  last.pt


# 6. tensorboard 로 학습이 잘 진행되었는지 확인

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# 7. car.mp4 동영상 detect하기

In [ ]:
!python detect.py --weights ./runs/train/car_results3/weights/last.pt --img 640 --conf 0.5 --source ../car.mp4

# 8. 검증하기(validation)
 val.py 사용

- weights : 검증에 사용할 학습된 모델(best.pt) 설정
- data : 데이터셋의 경로정보
- iou : 실측값과 예측값이 얼마나 겹치는지를 나타내는 척도(iou 0.65 : 0.65이하는 버린다.)


In [ ]:
!python val.py --weights runs/train/car_results3/weights/best.pt --data /content/car/data.yaml --img 640 --iou 0.65 --half